## Загрузка данных

In [12]:
import gensim
from gensim.models import Word2Vec
import re
import pymorphy2
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pickle
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
# from my_parsers import download, parse
from tqdm import tqdm
from collections import Counter
# from brown_clustering import BigramCorpus, BrownClustering
from util_functions import *
pd.options.display.max_rows = 200
tqdm.pandas()

In [2]:
df_TRAIN_RES_1 = pd.read_csv('vprod_train/TRAIN_RES_1.csv')

## Начальная предобработка данных

In [3]:
# удаляем модифицированный столбец - он совпадает с точностью до регистра
df_TRAIN_RES_1.drop("achievements_modified", axis=1, inplace=True)

for text_feature in ["achievements", "company_name", "demands"]:
    df_TRAIN_RES_1[text_feature].fillna("", inplace=True)

df_TRAIN_RES_1["achievements_len"] = [len(i) for i in df_TRAIN_RES_1["achievements"]]
df_TRAIN_RES_1.drop_duplicates(inplace=True)

In [5]:
DROP_PART_DUPLICATES = False

if DROP_PART_DUPLICATES:
    df_TRAIN_RES_1["comp_name_demands"] = df_TRAIN_RES_1["demands"] + df_TRAIN_RES_1["company_name"]
    df_TRAIN_RES_1 = df_TRAIN_RES_1.drop_duplicates(subset="comp_name_demands")
    df_TRAIN_RES_1.drop("comp_name_demands", axis=1, inplace=True)

In [8]:
vc = df_TRAIN_RES_1["job_title"].value_counts().sort_values(ascending=False)
print("proportion of samples choised clases: ", vc.iloc[:200].values.sum() / vc.values.sum())

proportion of samples choised clases:  0.9580346140836676


In [9]:
used_classes = df_TRAIN_RES_1["job_title"].value_counts().sort_values(ascending=False).iloc[:200].index.tolist()
df_TRAIN_RES_1 = df_TRAIN_RES_1[df_TRAIN_RES_1["job_title"].isin(used_classes)]
df_TRAIN_RES_1 = df_TRAIN_RES_1.iloc[::5]
df_TRAIN_RES_1["len_group"] = df_TRAIN_RES_1.groupby("id_cv").agg({"company_name": "count"})

In [10]:
df_TRAIN_RES_1.head()

,achievements,company_name,demands,id_cv,job_title,achievements_len,len_group
0,,Управление социальной защиты населения Сердобс...,"Работала,непосредственно, с федеральными льгот...",52860148-26da-11e8-a184-9122a281f90e,инспектор,0,NaN
5,,"ОАО «Высочайший» п.Бодайбо, Иркутская обл.",Разведка с применением поверхностных горных вы...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,0,NaN
10,,"ОАО «БАЙКАЛГЕО» п. Нижнеангарск, Бурятия",Проведение геологоразведочных работ на россыпн...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,заместитель,0,NaN
15,"нет, т.к. участкам дано отрицательное заключение","ООО «Артель старателей «ТАЛ»», Усть-Нера, Респ...",руководил поисково-разведочными работами на бу...,d09f8cf0-2b52-11e8-a7fa-ef76bd2a03c1,геолог,48,NaN
20,,"Администрация Северо-Байкальского района, п.Ни...","Разработка программ развития района, статистич...",d09f8cf0-2b52-11e8-a7fa-ef76bd2a03c1,заместитель,0,NaN


## w2v for company names

In [15]:

RETRAIN_MODEL = False

if RETRAIN_MODEL:
    company_names = [i["company_name"] + " " + i["job_title"] for _, i in df_TRAIN_RES_1.iterrows()]
    # Предобработка данных
    sentences = preprocess_names(company_names)

    # Обучение модели Word2Vec
    model_comp_name = Word2Vec(sentences, vector_size=64, window=4, min_count=1, sg=1)
    # Сохранение обученной модели
    model_comp_name.save("models/company_word2vec_russian.model")
else:
    company_names = [i["company_name"] for _, i in df_TRAIN_RES_1.iterrows()]
    # Предобработка данных
    sentences = preprocess_names(company_names)

    model_comp_name = Word2Vec.load("models/company_word2vec_russian.model")

# model_comp_name.wv.most_similar('компьютер')

100%|██████████| 318924/318924 [00:01<00:00, 172613.53it/s]


In [16]:
comp_names_without_job = preprocess_names(df_TRAIN_RES_1["company_name"])
emb_comp_name_df = get_emb_by_modele(model_comp_name, comp_names_without_job, column_prefix="comp_name_emb")
df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True), emb_comp_name_df.reset_index(drop=True)], axis=1)

100%|██████████| 318924/318924 [00:05<00:00, 61550.90it/s]


## w2v for demands

In [17]:
RETRAIN_MODEL = False

if RETRAIN_MODEL:
    company_names = [i["demands"] + " " + i["job_title"] for _, i in df_TRAIN_RES_1.iterrows()]
    # Предобработка данных
    sentences = preprocess_names(company_names)

    # Обучение модели Word2Vec
    model_demand = Word2Vec(sentences, vector_size=100, window=8, min_count=1, sg=1)
    # Сохранение обученной модели
    model_demand.save("models/demand_word2vec_russian.model")
else:
    company_names = [i["demands"] for _, i in df_TRAIN_RES_1.iterrows()]
    # Предобработка данных
    sentences = preprocess_names(company_names)
    model_demand = Word2Vec.load("models/demand_word2vec_russian.model")

model_demand.wv.most_similar('бухгалтер')

100%|██████████| 318924/318924 [00:04<00:00, 64405.87it/s]


[('авансовые', 0.8037424683570862),
 ('кассыбанка', 0.7973775863647461),
 ('бух', 0.7930706739425659),
 ('кассабанк', 0.7926923036575317),
 ('фсспфр', 0.7875778079032898),
 ('налоговойстатистической', 0.7757202386856079),
 ('клинтбанк', 0.7753341197967529),
 ('кудир', 0.7724557518959045),
 ('заработная', 0.7707493305206299),
 ('пфрфссифнс', 0.7690602540969849)]

In [18]:
demand_without_job = preprocess_names(df_TRAIN_RES_1["demands"])
emb_demand_df = get_emb_by_modele(model_demand, demand_without_job, column_prefix="demand_emb")
df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True), emb_demand_df.reset_index(drop=True)], axis=1)

100%|██████████| 318924/318924 [00:15<00:00, 20812.14it/s]


In [19]:
df_TRAIN_RES_1.head()

,achievements,company_name,demands,id_cv,job_title,achievements_len,len_group,comp_name_emb_0,comp_name_emb_1,comp_name_emb_2,...,demand_emb_90,demand_emb_91,demand_emb_92,demand_emb_93,demand_emb_94,demand_emb_95,demand_emb_96,demand_emb_97,demand_emb_98,demand_emb_99
0,,Управление социальной защиты населения Сердобс...,"Работала,непосредственно, с федеральными льгот...",52860148-26da-11e8-a184-9122a281f90e,инспектор,0,NaN,-0.293772,-0.564505,-0.419635,...,0.183418,0.258860,0.290902,-0.200651,0.256295,0.253528,0.045275,-0.140245,-0.149471,-0.227559
1,,"ОАО «Высочайший» п.Бодайбо, Иркутская обл.",Разведка с применением поверхностных горных вы...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,0,NaN,-0.330126,-0.029948,0.391855,...,0.170837,0.391514,0.734838,0.099061,0.517577,0.271957,0.088179,-0.515044,-0.201592,-0.200995
2,,"ОАО «БАЙКАЛГЕО» п. Нижнеангарск, Бурятия",Проведение геологоразведочных работ на россыпн...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,заместитель,0,NaN,-0.318120,0.166147,-0.214572,...,0.150262,0.246992,0.499546,-0.141324,0.373341,0.260797,0.153022,-0.454782,-0.204571,-0.216284
3,"нет, т.к. участкам дано отрицательное заключение","ООО «Артель старателей «ТАЛ»», Усть-Нера, Респ...",руководил поисково-разведочными работами на бу...,d09f8cf0-2b52-11e8-a7fa-ef76bd2a03c1,геолог,48,NaN,-0.201948,0.644514,-0.182316,...,0.123024,0.314913,0.608443,-0.120845,0.590910,0.153910,0.097952,-0.359757,-0.168976,-0.194691
4,,"Администрация Северо-Байкальского района, п.Ни...","Разработка программ развития района, статистич...",d09f8cf0-2b52-11e8-a7fa-ef76bd2a03c1,заместитель,0,NaN,-0.586076,-0.180409,-0.088037,...,0.084129,0.042485,0.603362,-0.176048,0.255731,0.096839,0.305328,-0.284165,-0.173182,-0.192775


## add tf_idf company name

In [21]:
sentences = preprocess_names(df_TRAIN_RES_1["company_name"])

RETRAIN_MODEL = False
column_prefix = "comp_name_tfidf"

if RETRAIN_MODEL:
    comp_name_vectorizer, comp_name_tfidf = get_emb_by_tfidf([" ".join(sent) for sent in sentences],
                                                             column_prefix=column_prefix)
else:
    comp_name_vectorizer = pickle.load(open("models/comp_name_vectorizer.pkl", "rb"))

    comp_name_tfidf = comp_name_vectorizer.transform([" ".join(sent) for sent in sentences])
    comp_name_tfidf = pd.DataFrame(comp_name_tfidf.toarray(),
                                   columns=[f"{column_prefix}_{word}" for word in comp_name_vectorizer.get_feature_names_out()])

df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True),
                            comp_name_tfidf.reset_index(drop=True)], axis=1)

100%|██████████| 318924/318924 [00:01<00:00, 173266.69it/s]


In [22]:
sentences = preprocess_names(df_TRAIN_RES_1["demands"])

RETRAIN_MODEL = False
column_prefix = "demnds_tfidf"

if RETRAIN_MODEL:
    demands_vectorizer, demands_tfidf = get_emb_by_tfidf([" ".join(sent) for sent in sentences],
                                                     column_prefix=column_prefix)
else:
    demands_vectorizer = pickle.load(open("models/demands_vectorizer.pkl", "rb"))

    demands_tfidf = demands_vectorizer.transform([" ".join(sent) for sent in sentences])
    demands_tfidf = pd.DataFrame(demands_tfidf.toarray(),
                                 columns=[f"{column_prefix}_{word}" for word in demands_vectorizer.get_feature_names_out()])

df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True),
                            demands_tfidf.reset_index(drop=True)], axis=1)

100%|██████████| 318924/318924 [00:07<00:00, 40739.31it/s]


In [23]:
sentences = preprocess_names(df_TRAIN_RES_1["achievements"])

RETRAIN_MODEL = False
column_prefix = "achiv_tfidf"

if RETRAIN_MODEL:
    achiv_vectorizer, achiv_tfidf = get_emb_by_tfidf([" ".join(sent) for sent in sentences],
                                                     column_prefix=column_prefix)
else:
    achiv_vectorizer = pickle.load(open("models/achiv_vectorizer.pkl", "rb"))
    achiv_tfidf = achiv_vectorizer.transform([" ".join(sent) for sent in sentences])
    achiv_tfidf = pd.DataFrame(achiv_tfidf.toarray(),
                               columns=[f"{column_prefix}_{word}" for word in demands_vectorizer.get_feature_names_out()])

df_TRAIN_RES_1 = pd.concat([df_TRAIN_RES_1.reset_index(drop=True),
                            achiv_tfidf.reset_index(drop=True)], axis=1)

100%|██████████| 318924/318924 [00:03<00:00, 93251.18it/s] 


In [24]:
df_TRAIN_RES_1.head()

,achievements,company_name,demands,id_cv,job_title,achievements_len,len_group,comp_name_emb_0,comp_name_emb_1,comp_name_emb_2,...,achiv_tfidf_техники,achiv_tfidf_товара,achiv_tfidf_товаров,achiv_tfidf_труда,achiv_tfidf_управление,achiv_tfidf_услуг,achiv_tfidf_участие,achiv_tfidf_учет,achiv_tfidf_учета,achiv_tfidf_формирование
0,,Управление социальной защиты населения Сердобс...,"Работала,непосредственно, с федеральными льгот...",52860148-26da-11e8-a184-9122a281f90e,инспектор,0,NaN,-0.293772,-0.564505,-0.419635,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,,"ОАО «Высочайший» п.Бодайбо, Иркутская обл.",Разведка с применением поверхностных горных вы...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,геолог,0,NaN,-0.330126,-0.029948,0.391855,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,,"ОАО «БАЙКАЛГЕО» п. Нижнеангарск, Бурятия",Проведение геологоразведочных работ на россыпн...,e3c857b0-26a0-11e7-bcca-736ab11edb0c,заместитель,0,NaN,-0.318120,0.166147,-0.214572,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"нет, т.к. участкам дано отрицательное заключение","ООО «Артель старателей «ТАЛ»», Усть-Нера, Респ...",руководил поисково-разведочными работами на бу...,d09f8cf0-2b52-11e8-a7fa-ef76bd2a03c1,геолог,48,NaN,-0.201948,0.644514,-0.182316,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,,"Администрация Северо-Байкальского района, п.Ни...","Разработка программ развития района, статистич...",d09f8cf0-2b52-11e8-a7fa-ef76bd2a03c1,заместитель,0,NaN,-0.586076,-0.180409,-0.088037,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# with open("comp_name_vectorizer.pkl", "wb") as f:
#     pickle.dump(comp_name_vectorizer, f)
#
# with open("demands_vectorizer.pkl", "wb") as f:
#     pickle.dump(demands_vectorizer, f)
#
# with open("achiv_vectorizer.pkl", "wb") as f:
#     pickle.dump(achiv_vectorizer, f)


## train model

In [25]:
columns_to_drop = ["achievements", "company_name", "demands", "id_cv",]

In [26]:
X, y = df_TRAIN_RES_1.drop(columns_to_drop + ["job_title"], axis=1), df_TRAIN_RES_1["job_title"]

In [27]:
from sklearn.model_selection import train_test_split
N = 20_000
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:N], y.iloc[:N], stratify=y.iloc[:N], test_size=0.2, random_state=42)

In [28]:
from catboost import CatBoostClassifier, Pool

train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

In [29]:
clf = CatBoostClassifier(task_type="GPU", eval_metric="Accuracy")
clf.fit(train_pool, eval_set=test_pool, early_stopping_rounds=5)

Learning rate set to 0.139139
0:	learn: 0.2304375	test: 0.2275000	best: 0.2275000 (0)	total: 932ms	remaining: 15m 31s
1:	learn: 0.2476250	test: 0.2507500	best: 0.2507500 (1)	total: 1.8s	remaining: 14m 58s
2:	learn: 0.2646250	test: 0.2672500	best: 0.2672500 (2)	total: 2.7s	remaining: 14m 56s
3:	learn: 0.2741875	test: 0.2722500	best: 0.2722500 (3)	total: 3.6s	remaining: 14m 57s
4:	learn: 0.2837500	test: 0.2815000	best: 0.2815000 (4)	total: 4.51s	remaining: 14m 58s
5:	learn: 0.2931875	test: 0.2907500	best: 0.2907500 (5)	total: 5.42s	remaining: 14m 57s
6:	learn: 0.2999375	test: 0.2965000	best: 0.2965000 (6)	total: 6.2s	remaining: 14m 39s
7:	learn: 0.3068750	test: 0.3002500	best: 0.3002500 (7)	total: 7.11s	remaining: 14m 41s
8:	learn: 0.3155000	test: 0.3080000	best: 0.3080000 (8)	total: 7.95s	remaining: 14m 34s
9:	learn: 0.3204375	test: 0.3142500	best: 0.3142500 (9)	total: 8.84s	remaining: 14m 35s
10:	learn: 0.3274375	test: 0.3212500	best: 0.3212500 (10)	total: 9.69s	remaining: 14m 31s
11:	

In [39]:
with open("clf.pkl", "wb") as f:
    pickle.dump(clf, f)

In [47]:
from collections import Counter
Counter([i[0] for i in clf.predict(X_test).tolist()])

Counter({'менеджер': 342,
         'электрогазосварщик': 14,
         'оператор': 223,
         'водитель': 130,
         'юрисконсульт': 91,
         'специалист': 686,
         'кладовщик': 38,
         'директор': 154,
         'мастер': 108,
         'администратор': 87,
         'кассир': 40,
         'слесарь': 93,
         'психолог': 14,
         'инженер': 444,
         'бухгалтер': 287,
         'руководитель': 42,
         'продавец': 94,
         'машинист': 67,
         'электромонтер': 73,
         'заместитель': 99,
         'экономист': 72,
         'фельдшер': 11,
         'консультант': 15,
         'инспектор': 54,
         'секретарь': 77,
         'архивариус': 1,
         'учитель': 77,
         'сборщик': 4,
         'механик': 34,
         'укладчик': 2,
         'медсестра': 7,
         'официант': 8,
         'монтажник': 37,
         'педагог': 14,
         'преподаватель': 76,
         'охранник': 23,
         'юрист': 22,
         'воспитатель': 42,
       